# Pipelines

In [1]:
import pandas as pd
pd.set_option('display.max_rows', 80) # esta opcion podria ser necesario cambiarla a lo largo de la notebook

In [2]:
# Read the data
X_full = pd.read_csv('train.csv', index_col='Id')
X_test_full = pd.read_csv('test.csv', index_col='Id')

X_full.shape, X_test_full.shape

((1460, 80), (1459, 79))

Tenemos que eliminar las filas que tienen datos faltantes en la columna `SalePrice`, porque no nos van a servir ni para entrenar el modelo ni para validarlo

In [3]:
# eliminamos las filas que tienen datos faltantes en la columna SalePrice
X_full.dropna(axis = 0, subset=['SalePrice'], inplace = True)
X_full.shape

(1460, 80)

In [4]:
# definimos las etiquetas que vamos utilizar para entrenar y validar el modelo
y = X_full.SalePrice
y[:10]

Id
1     208500
2     181500
3     223500
4     140000
5     250000
6     143000
7     307000
8     200000
9     129900
10    118000
Name: SalePrice, dtype: int64

In [5]:
# eliminamos la columna SalePrice de los datos de entrenamiento
X_full.drop(['SalePrice'], axis = 1, inplace = True)
X_full.shape

(1460, 79)

In [6]:
# Hacemos la division en datos de entrenamiento y validacion
from sklearn.model_selection import train_test_split

X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_full, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

In [7]:
X_train_full.shape, y_train.shape

((1168, 79), (1168,))

In [8]:
X_valid_full.shape, y_valid.shape

((292, 79), (292,))

In [9]:
# Que tipos de datos tenemos almacenados en la columnas de los datos de entrenamiento y validacion
X_train_full.dtypes

MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
Street            object
Alley             object
LotShape          object
LandContour       object
Utilities         object
LotConfig         object
LandSlope         object
Neighborhood      object
Condition1        object
Condition2        object
BldgType          object
HouseStyle        object
OverallQual        int64
OverallCond        int64
YearBuilt          int64
YearRemodAdd       int64
RoofStyle         object
RoofMatl          object
Exterior1st       object
Exterior2nd       object
MasVnrType        object
MasVnrArea       float64
ExterQual         object
ExterCond         object
Foundation        object
BsmtQual          object
BsmtCond          object
BsmtExposure      object
BsmtFinType1      object
BsmtFinSF1         int64
BsmtFinType2      object
BsmtFinSF2         int64
BsmtUnfSF          int64
TotalBsmtSF        int64
Heating           object
HeatingQC         object


In [10]:
# vamos a seleccionar las columnas que tiene datos categoricos y tienen una cardinalidad menor a 10
# recordemos que cardinalidad en el numero de categorias unicas que tiene una columna


categorical_cols = [cname for cname in X_train_full.columns if
                  X_train_full[cname].nunique() < 10 and
                  X_train_full[cname].dtype == 'object']
categorical_cols

['MSZoning',
 'Street',
 'Alley',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'PoolQC',
 'Fence',
 'MiscFeature',
 'SaleType',
 'SaleCondition']

In [11]:
# ademas vamosa  seleccionar las columnas que tienen datos numercicos

numerical_cols = [col for col in X_train_full.columns if
                 X_train_full[col].dtype in ['int64', 'float']]
numerical_cols

['MSSubClass',
 'LotFrontage',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageYrBlt',
 'GarageCars',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'MoSold',
 'YrSold']

In [12]:
# nos vamos a quedar solo con las columnas seleccionadas antes .
# las que estan almacenadas en las listas categorical_cols y numerical_cols

my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

X_train.shape, X_valid.shape, X_test.shape

((1168, 76), (292, 76), (1459, 76))

In [13]:
# Librerias
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [14]:
# Preprocesamiento para los datos numericos
#  -Vamos a imputar los datos numericos faltantes con SimpleImputer(strategy='constant') 
numerical_transformer = SimpleImputer(strategy='constant')

#-------------------------------------------------------------------------------------------------

# Preprocesamiento para los datos categoricos
#  -Vamos a imputar los datos categoricos faltantes con SimpleImputer(strategy='most_frequent')
#  -Vamos a codificar los datos categoricos con OneHotEncoder(handle_unknown='ignore'
categorical_transformer = Pipeline(steps = [
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

#-------------------------------------------------------------------------------------------------

# Tratamiento de datos numericos y categoricos
preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_transformer, numerical_cols),
    ('cat', categorical_transformer, categorical_cols)
])

#-------------------------------------------------------------------------------------------------

#Definimos el modelo
model = RandomForestRegressor(n_estimators=100, random_state=0)

#-------------------------------------------------------------------------------------------------

# Armamos una Pipeline para correr el preprocesamiento y el modelo
clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])

#-------------------------------------------------------------------------------------------------

# Preprocesamiento de los datos de entrenamiento y validacion
# Tambien puede ser entendidio como el ENTRENEMIENTO  de la pipeline clf
clf.fit(X_train, y_train)

#-------------------------------------------------------------------------------------------------

#Preprocesamiento de los datos de validacion y prediciones
predictions = clf.predict(X_valid)

#-------------------------------------------------------------------------------------------------

#Resultados
print('-'*50, 'RESULTS','-'*50)
print('MAE:', mean_absolute_error(y_valid, predictions))
print('-'*50, 'RESULTS','-'*50)


-------------------------------------------------- RESULTS --------------------------------------------------
MAE: 17861.780102739725
-------------------------------------------------- RESULTS --------------------------------------------------


Tal como estan seteados los parametros de las funciones anteriores obtuvimos como resultado umn MAE de 17861.<br>
La estrategia de Pipelines nos permite hacer una analisis rapido ajustando diferentes parametros y evaluar cuando de los resultados es mejor.

### Ajuste de parametros

In [27]:
# Preprocesamiento para los datos numericos
numerical_transformer = SimpleImputer(strategy='median') 

# Preprocesamiento para los datos categoricos
categorical_transformer = Pipeline(steps = [
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Tratamiento de datos numericos y categoricos
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

#Definimos el modelo
model = RandomForestRegressor(n_estimators=500, random_state=0) # Your code here

In [28]:
# Armamos una Pipeline para correr el preprocesamiento y el modelo
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Preprocesamiento de los datos de entrenamiento y validacion
my_pipeline.fit(X_train, y_train)

#Preprocesamiento de los datos de validacion y prediciones
preds = my_pipeline.predict(X_valid)

#Resultados
print('-'*50, 'RESULTS','-'*50)
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)
print('-'*50, 'RESULTS','-'*50)

-------------------------------------------------- RESULTS --------------------------------------------------
MAE: 17413.575452054793
-------------------------------------------------- RESULTS --------------------------------------------------


Modificando algunos parametros de los algoritmos de imputacion y el modelo logramos reducir un poco el error absoluto medio.

### Prediccion
Podemos utilizar la misma Pipeline que creamos antes para predecir los valores de `SalePrice` de los datos de test

In [34]:
predictions = my_pipeline.predict(X_test)
len(predictions)

1459

In [35]:
len(X_test.index)

1459

In [36]:
# Save test predictions to file
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': predictions})
output.to_csv('submission-04-Pipelines.csv', index=False)

In [37]:
output.head()

,Id,SalePrice
0,1461,128892.216
1,1462,153965.754
2,1463,184577.248
3,1464,182344.510
4,1465,199404.462
